<h1> Custom Environment </h1>
<h4> -An attempt at creating it from scratch </h4>
<i> "You either die a hero, or live long enough to see yourself become a villain."</i>

class
<br>
initializer
<br>
step
<br>
reset
<br>
render
<br>
close
<br>
<br>
Validate thorugh check_env()

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb#scrollTo=9DOpP_B0-LXm

In [ ]:
import numpy as np
import pandas as pd

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import SAC
from stable_baselines3 import PPO
from stable_baselines3 import RecurrentPPO

In [2]:
history_usage = 30

return_data = pd.read_csv("../../Data/StockReturns.csv")

esg_data = np.array([36.6, 35.3, 17.9, 18, 
                    18, 21.2, 18.7, 29.2, 
                    15.7, 25.6, 25.6, 18.4, 
                    19.8, 13.8, 18.1, 19, 
                    17.2, 14, 17.2, 19.5, 
                    19.7, 21.2, 26.8, 19.3])

objective = "Sharpe"

esg_compliancy = True

In [3]:
split_size = 0.8

train_data = return_data.iloc[:int(split_size*len(return_data))]
test_data = return_data.iloc[int(split_size*len(return_data)):].reset_index(drop=True)

In [32]:
class PortfolioEnvironment(gym.Env):
    """
    doc string
    """
    def __init__(self,
                 history_usage,
                 return_data, esg_data,
                 objective, esg_compliancy):
        super().__init__()
        """
        doc  string,

        Good, initialize all variables with values 
        """
        self.return_data = return_data.values
        self.esg_data: np.array = esg_data
        self.history_usage: int = history_usage
        self.n_stocks = len(esg_data)

        self.objective: str = objective
        self.esg_compliancy: bool = esg_compliancy

        self.action_space = spaces.Box(low=-1, 
                                       high=1, 
                                       shape=(self.n_stocks,),)
        self.observation_space = spaces.Box(low=-np.inf, 
                                            high=np.inf, 
                                            shape=(self.n_stocks * self.history_usage,))

        self.current_step: int = 0
        self.weights_list: list = []
        self.returns_list: list = []
        


    def reset(self, seed=42):
        """
        doc string

        Good, changing all non-fixed variables inside the environment
        """
        super().reset(seed=None)

        self.current_step = 0
        self.weights = []
        self.portfolio_returns = []

        observation = self.get_observation()
        additional_info = {
            "time_step": self.current_step,
            "cumulative_geo_return": np.cumprod(self.portfolio_returns)
        }

        return observation, additional_info



    def get_observation(self):
        """
        doc string
        """
        start_idx = max(0, self.current_step -self.history_usage)
        end_idx = self.current_step
        observation_space  = self.return_data[start_idx:end_idx].T
        if observation_space.shape[1] < self.history_usage:
            padding = np.zeros((self.n_stocks, self.history_usage - observation_space.shape[1]))
            observation_space = np.hstack([padding, observation_space])
        
        return observation_space.flatten().astype(np.float32)


    def step(self, action):
        """
        doc string
        """
        # Generate weights based on actions
        # Forces action from in range (-1,1) to become (0,1)
        current_weights = (action + 1) / 2                          
        current_weights = (action+1e-8) / (np.sum(action)+1e-8)
        self.weights_list.append(current_weights)
        
        # Find current weights and multiply with weights
        # Variables for (early) stopping
        terminated = self.current_step >= len(self.return_data)-1
        truncated = False
        if not terminated:
            current_returns = self.return_data[self.current_step +1]
            portfolio_return = 0.0
            if self.current_step +1 < len(self.return_data):
                portfolio_return = np.dot(current_weights, current_returns)
            self.returns_list.append(portfolio_return)
        else:
            portfolio_return = 0.0
            self.returns_list.append(portfolio_return)
        # New step
        self.current_step += 1

        # if terminated:
            

        # Returns the next observation space for the algo to use
        next_window = self.get_observation()

        return next_window, portfolio_return, terminated, truncated, {}
        


    def render(self, mode="human"):
        """ 
        doc string
        """
        print(f"Current step: {self.current_step}, and geometric return: {np.cumprod(self.portfolio_returns)}")
        pass

In [33]:
train_env = PortfolioEnvironment(history_usage=history_usage, 
                              return_data=train_data,
                              esg_data=esg_data,
                              objective=objective,
                              esg_compliancy=esg_compliancy)
check_env(train_env, warn=True)

In [ ]:
model = SAC(policy="MlpPolicy",
            env=train_env,
            buffer_size=100_000,
            verbose=1
            ).learn(35000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.82e+03 |
|    ep_rew_mean     | 93.1     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 30       |
|    time_elapsed    | 641      |
|    total_timesteps | 19268    |
| train/             |          |
|    actor_loss      | -208     |
|    critic_loss     | 5.06     |
|    ent_coef        | 0.0799   |
|    ent_coef_loss   | 19.2     |
|    learning_rate   | 0.0003   |
|    n_updates       | 19167    |
---------------------------------


In [ ]:
test_env = PortfolioEnvironment(history_usage=history_usage, 
                              return_data=test_data,
                              esg_data=esg_data,
                              objective=objective,
                              esg_compliancy=esg_compliancy)

In [ ]:
obs, info = test_env.reset()
weights_history = []
finished = False

while not finished: 
    action, _ = model.predict(obs, deterministic=True)

    weights = (action+1) / 2
    weights /= np.sum(weights)

    obs, reward, terminated, truncated, info = test_env.step(action)
    finished = terminated or truncated

    weights_history.append(weights)

In [ ]:
df = pd.DataFrame(weights_history)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.001610,0.073689,0.049533,0.001476,0.020851,0.001901,0.072798,0.073802,0.064445,0.071762,...,0.072030,0.007752,0.072667,0.067277,0.001894,0.004220,0.070938,0.018059,0.071649,0.042774
1,0.001458,0.073727,0.050801,0.001538,0.020463,0.002090,0.072814,0.073724,0.063672,0.071759,...,0.072172,0.007458,0.072630,0.067028,0.002064,0.003888,0.071049,0.018463,0.071732,0.041679
2,0.001618,0.073907,0.048802,0.001252,0.019687,0.001693,0.073304,0.074100,0.064952,0.072070,...,0.072188,0.006692,0.072950,0.068100,0.001377,0.004067,0.070711,0.016870,0.071714,0.045926
3,0.002040,0.073385,0.049175,0.001323,0.021346,0.001569,0.072931,0.073761,0.066168,0.071693,...,0.071657,0.006465,0.072662,0.068111,0.001094,0.004078,0.069563,0.015579,0.071251,0.047245
4,0.001799,0.073321,0.050279,0.001450,0.021090,0.001604,0.072746,0.073706,0.066003,0.071479,...,0.071791,0.007387,0.072571,0.068221,0.001244,0.004218,0.069920,0.016733,0.071383,0.044332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.000130,0.073647,0.064546,0.002745,0.010086,0.015389,0.072794,0.072054,0.028233,0.071718,...,0.073064,0.023109,0.072302,0.060754,0.022693,0.007488,0.072981,0.048344,0.072792,0.019645
1201,0.005721,0.074325,0.040385,0.000345,0.013976,0.000379,0.075617,0.076152,0.072972,0.074292,...,0.066864,0.003059,0.075608,0.072285,0.000047,0.005024,0.064087,0.008400,0.066930,0.068502
1202,0.009905,0.068240,0.063276,0.004662,0.049462,0.000548,0.060876,0.067957,0.067806,0.068426,...,0.064480,0.014445,0.066124,0.067590,0.000398,0.002868,0.062893,0.005440,0.067359,0.051548
1203,0.000278,0.074142,0.057083,0.001353,0.021963,0.004830,0.073108,0.073686,0.049042,0.072671,...,0.073647,0.005060,0.072691,0.065161,0.009970,0.001787,0.073560,0.025437,0.073693,0.022490
